In [1]:
import os

from platform import python_version
import warnings
import time
import datetime as dt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import multiprocessing as mp
import shutil

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input, decode_predictions
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras import regularizers

import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from PIL import Image
import xml.etree.ElementTree as ET
import psutil
import random

warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
img_size = 96
epochs = 75
batch_size = 32
testsplit = .2
targetx = 96
targety = 96
learning_rate = 0.00001
classes = 7
seed = 23
print(seed)


23


In [3]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from collections import Counter

model_name = 'MNFemaleDominantmodel.keras'
# Define two different ImageDataGenerators with their respective directories
datagen1 = ImageDataGenerator(
        rescale=1./255,
        brightness_range=[0.9,1.1],
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=preprocess_input
)

datagen2 = ImageDataGenerator(
        rescale=1./255,
        brightness_range=[0.9,1.1],
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=preprocess_input
)

train_dir1 = 'KDEFMF/MaleNonDominantTraining'
train_dir2 = 'KDEFMF/FemaleDominantTraining'
val_dir1 = 'KDEFMF/MaleBalancedValidation'
val_dir2 = 'KDEFMF/FemaleBalancedValidation'

# Create two separate generators using different directories
train_generator1 = datagen1.flow_from_directory(
    train_dir1,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=seed
)
validation_generator1 = datagen1.flow_from_directory(
    val_dir1,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical'
)
validation_generator2 = datagen2.flow_from_directory(
    val_dir2,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical'
)

train_generator2 = datagen2.flow_from_directory(
    train_dir2,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=seed
)

total_train_samples = len(train_generator1.filenames) + len(train_generator2.filenames)
total_validation_samples = len(validation_generator1.filenames) + len(validation_generator2.filenames)

# Combine labels from both generators
all_labels = list(train_generator1.classes) + list(train_generator2.classes)

# Get class indices (assuming they are the same for both generators)
class_indices = train_generator1.class_indices
index_to_class = {v: k for k, v in class_indices.items()}

# Count the number of occurrences of each class
class_counts = Counter(all_labels)

# Print the number of images per class
for class_index, count in class_counts.items():
    class_name = index_to_class[class_index]
    print(f"Class '{class_name}' has {count} images.")
    
def combined_generator(gen1, gen2, num_batches):
    while True:
        try:
            batch1_x, batch1_y = next(gen1)
            batch2_x, batch2_y = next(gen2)

            combined_x = np.concatenate((batch1_x, batch2_x), axis=0)
            combined_y = np.concatenate((batch1_y, batch2_y), axis=0)

            yield combined_x, combined_y
        except StopIteration as e:
            print("Generator exhausted or encountered an error:", e)
            break


Found 420 images belonging to 7 classes.
Found 280 images belonging to 7 classes.
Found 280 images belonging to 7 classes.
Found 980 images belonging to 7 classes.
Class 'angry' has 200 images.
Class 'disgust' has 200 images.
Class 'fear' has 200 images.
Class 'happy' has 200 images.
Class 'neutral' has 200 images.
Class 'sad' has 200 images.
Class 'surprise' has 200 images.


In [4]:
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(targetx, targety, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten() (x)
x = BatchNormalization()(x)
x = Dense(2048, activation='relu', kernel_initializer='glorot_uniform',
          kernel_regularizer=regularizers.L2(0.001))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(1024, activation='relu', kernel_initializer=glorot_uniform(seed=23),
          kernel_regularizer=regularizers.L2(0.001))(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x=Dropout(0.4)(x)
x = Dense(512, activation='relu',  kernel_initializer=glorot_uniform(seed),kernel_regularizer=regularizers.L2(0.001),activity_regularizer=regularizers.L2(1e-4), bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x=Dropout(0.4)(x)
x = Dense(256, activation='relu',  kernel_initializer=glorot_uniform(seed),kernel_regularizer=regularizers.L2(0.001),activity_regularizer=regularizers.L2(1e-4), bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x=Dropout(0.4)(x)
x = Dense(128, activation='relu',  kernel_initializer=glorot_uniform(seed),kernel_regularizer=regularizers.L2(0.001),activity_regularizer=regularizers.L2(1e-4), bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x=Dropout(0.4)(x)
predictions = Dense(classes, activation='softmax', kernel_initializer='random_uniform', bias_initializer='zeros')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=predictions)
optimizer = Adam(learning_rate=learning_rate)
loss = "categorical_crossentropy"

for layer in model.layers:
    layer.trainable = True

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=["accuracy"])

In [5]:
# Define the number of batches you want from each generator
num_batches_per_gen = 100  # Adjust as needed

# Use the combined generator for training
combined_train_generator = combined_generator(train_generator1, train_generator2, num_batches_per_gen)

# Use the combined generator for validation
combined_validation_generator = combined_generator(validation_generator1, validation_generator2, num_batches_per_gen)


steps_per_epoch = total_train_samples // (2 * 32)
validation_steps = total_validation_samples // (2 * 32)

# Ensure correct optimizer and loss function
optimizer = Adam(learning_rate=0.00001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Ensure ModelCheckpoint is correctly configured used for callback to save the best value based on val_accuracy
checkpoint = ModelCheckpoint(model_name, monitor='val_accuracy', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

with tf.device('/GPU:0'):
# Train the model with callbacks
    history = model.fit(
        combined_train_generator,
        steps_per_epoch=steps_per_epoch,
        validation_data=combined_validation_generator,
        validation_steps=validation_steps,
        epochs=epochs,
        callbacks=[checkpoint]
    )

Epoch 1/75
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 772ms/step - accuracy: 0.1215 - loss: 14.8829
Epoch 1: val_accuracy improved from -inf to 0.14258, saving model to MNFemaleDominantmodel.keras
21/21 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.1219 - loss: 14.8772 - val_accuracy: 0.1426 - val_loss: 6.4394
Epoch 2/75
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 752ms/step - accuracy: 0.1423 - loss: 14.6445
Epoch 2: val_accuracy improved from 0.14258 to 0.15927, saving model to MNFemaleDominantmodel.keras
21/21 ━━━━━━━━━━━━━━━━━━━━ 20s 954ms/step - accuracy: 0.1422 - loss: 14.6370 - val_accuracy: 0.1593 - val_loss: 6.4407
Epoch 3/75
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 713ms/step - accuracy: 0.1270 - loss: 14.8665
Epoch 3: val_accuracy did not improve from 0.15927
21/21 ━━━━━━━━━━━━━━━━━━━━ 17s 825ms/step - accuracy: 0.1275 - loss: 14.8569 - val_accuracy: 0.1411 - val_loss: 6.5385
Epoch 4/75
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 656ms/step - accuracy: 0.1376 - loss: 14.6024
Epoch 4: val_accuracy did not improve from 0.15927

In [6]:
model = tf.keras.models.load_model(model_name)

In [7]:
# Define the number of batches you want from each generator
num_batches_per_gen = 100  # Adjust as needed

# Use the combined generator for training
combined_train_generator = combined_generator(train_generator1, train_generator2, num_batches_per_gen)

# Use the combined generator for validation
combined_validation_generator = combined_generator(validation_generator1, validation_generator2, num_batches_per_gen)


steps_per_epoch = total_train_samples // (2 * 32)
validation_steps = total_validation_samples // (2 * 32)

In [8]:
# Collect predictions and true labels
y_true = []
y_pred = []

for batch_x, batch_y in combined_validation_generator:
    # Make predictions
    predictions = model.predict(batch_x)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(batch_y, axis=1)
    
    # Append to lists
    y_true.extend(true_classes)
    y_pred.extend(predicted_classes)
    
    # Stop if you have processed the entire validation set
    if len(y_true) >= total_validation_samples:
        break

# Generate classification report
report = classification_report(y_true, y_pred, target_names=validation_generator1.class_indices.keys())
print(report)

2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 6s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
              precision    recall  f1-score   support

       angry       0.17      0.72      0.27        74
     disgust       0.00      0.00      0.00        83
        fear       0.14      0.47      0.22        75
       happy       0.00      0.00      0.00        82
     neutral       0.00      0.00      0.00        77
         sad       0.00      0.00      0.00        85
    surprise       0.00      0.00      0.00        84

    accuracy                           0.16       560
   macro avg       0.04      0.17      0.07       560
weighted avg       0.04      0.16      0.07       560



In [9]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

# Assuming you have the model and validation generators defined
# validation_generator1 and validation_generator2
# Also assuming the target_names are the same for both generators

# Function to generate and print classification report
def generate_classification_report(generator, generator_name):
    y_true = []
    y_pred = []

    for batch_x, batch_y in generator:
        # Generate predictions
        predictions = model.predict(batch_x)
        predicted_classes = np.argmax(predictions, axis=1)
        true_classes = np.argmax(batch_y, axis=1)

        # Collect predictions and true labels
        y_true.extend(true_classes)
        y_pred.extend(predicted_classes)

        # Break if processed all samples
        if len(y_true) >= generator.samples:
            break

    # Generate and print the classification report
    target_names = list(generator.class_indices.keys())
    report = classification_report(y_true, y_pred, target_names=target_names)
    print(f"Classification Report for {generator_name}:\n{report}")

# Generate and print classification reports for both validation generators
generate_classification_report(validation_generator1, "Validation Generator 1")
generate_classification_report(validation_generator2, "Validation Generator 2")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Classification Report for Validation Generator 1:
              precision    recall  f1-score   support

       angry       0.16      0.72      0.26        39
     disgust       0.00      0.00      0.00        36
        fear       0.18      0.42      0.26        45
       happy       0.00      0.00      0.00        40
     neutral       0.00      0.00      0.00        40
         sad       0.00      0.00      0.00        41
    surprise       0.00      0.00      0.00        39

    accuracy                           0.17       280
   macro avg       0.05      0.16      0.07       280
weighted avg       0.05      0.17      0.08       280

1/1

In [10]:
predictions = model.predict(validation_generator1, steps=len(validation_generator1))
y = np.argmax(predictions, axis=1)

print('Classification Report')
cr = classification_report(y_true=validation_generator1.classes, y_pred=y, target_names=validation_generator1.class_indices)
print(cr)

9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 588ms/step
Classification Report
              precision    recall  f1-score   support

       angry       0.18      0.72      0.29        40
     disgust       0.00      0.00      0.00        40
        fear       0.14      0.42      0.22        40
       happy       0.00      0.00      0.00        40
     neutral       0.00      0.00      0.00        40
         sad       0.00      0.00      0.00        40
    surprise       0.00      0.00      0.00        40

    accuracy                           0.16       280
   macro avg       0.05      0.16      0.07       280
weighted avg       0.05      0.16      0.07       280



In [11]:
Y_pred = model.predict(validation_generator1)
y_pred = np.argmax(Y_pred, axis=1)
y_true=validation_generator1.classes

Y_pred1 = model.predict(validation_generator2)
y_pred1 = np.argmax(Y_pred1, axis=1)
y_true1=validation_generator2.classes

cm_first = confusion_matrix(y_true, y_pred)

cm_second = confusion_matrix(y_true1, y_pred1)

# Print the confusion matrices
print("Confusion Matrix for First Dataset:")
print(cm_first)

print("Confusion Matrix for Second Dataset:")
print(cm_second)

9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 382ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 286ms/step
Confusion Matrix for First Dataset:
[[23  0 17  0  0  0  0]
 [20  0 20  0  0  0  0]
 [18  0 22  0  0  0  0]
 [22  0 18  0  0  0  0]
 [27  0 13  0  0  0  0]
 [27  0 13  0  0  0  0]
 [22  0 18  0  0  0  0]]
Confusion Matrix for Second Dataset:
[[23  0 17  0  0  0  0]
 [17  0 23  0  0  0  0]
 [19  0 21  0  0  0  0]
 [20  0 20  0  0  0  0]
 [18  0 22  0  0  0  0]
 [23  0 17  0  0  0  0]
 [20  0 20  0  0  0  0]]


In [14]:
from keras import metrics

accuracy_male = metrics.Accuracy()
accuracy_female = metrics.Accuracy()
accuracy_male.update_state(y_true, y_pred)
accuracy_female.update_state(y_true1, y_pred1)

# Calculate the accuracy results
accuracy_result_male = accuracy_male.result().numpy()
accuracy_result_female = accuracy_female.result().numpy()

# Print the individual accuracies
print("Male Accuracy: %f" % accuracy_result_male)
print("Female Accuracy: %f" % accuracy_result_female)

# Calculate the accuracy difference
accuracy_difference = accuracy_result_male - accuracy_result_female
DP = accuracy_result_female/accuracy_result_male
# Print the accuracy difference
print("Accuracy Difference: %f" % accuracy_difference)
print("DP: %f" % DP)

Male Accuracy: 0.160714
Female Accuracy: 0.157143
Accuracy Difference: 0.003571
DP: 0.977778
